### Structured Streaming File Source 프로그램 구조
1. 디렉토리로부터 스트리밍 데이터 파일을 읽어 DataFrame을 생성
- Streaming DataFrame이라고 함
- static DataFrame: Streaming DataFrame이 아닌 일반 DataFrame
2. DataFrame 작업 실행
- Static DataFrame의 자체 methods 실행과 동일
3. 결과 DataFrame의 산출 쿼리를 실행
4. 스트리밍 작업 대기

In [1]:
import findspark

In [2]:
findspark.init()

In [3]:
import pyspark

In [4]:
import pyspark.sql

In [5]:
from pyspark.sql.functions import *

In [6]:
spark = pyspark.sql.SparkSession \
.builder \
.master("local") \
.getOrCreate()

In [12]:
dessertMenuDF = spark.read \
.option("inferSchema", "true") \
.option("header", "true") \
.format("csv") \
.load("C:/spark/data/dessert-menu.csv")

In [13]:
dessertMenuDF.show()

+------+---------------+-----+----+
|menuId|           name|price|kcal|
+------+---------------+-----+----+
|   D-0|  초콜릿 파르페| 4900| 420|
|   D-1|    푸딩 파르페| 5300| 380|
|   D-2|    딸기 파르페| 5200| 320|
|   D-3|       판나코타| 4200| 283|
|   D-4|      치즈 무스| 5800| 288|
|   D-5|       아포가토| 3000| 248|
|   D-6|       티라미스| 6000| 251|
|   D-7|    녹차 파르페| 4500| 380|
|   D-8|  바닐라 젤라또| 3600| 131|
|   D-9|  카라멜 팬케익| 3900| 440|
|  D-10|    크림 안미츠| 5000| 250|
|  D-11|  고구마 파르페| 6500| 650|
|  D-12|      녹차 빙수| 3800| 320|
|  D-13|  초코 크레이프| 3700| 300|
|  D-14|바나나 크레이프| 3300| 220|
|  D-15|  커스터드 푸딩| 2000| 120|
|  D-16|    초코 토르테| 3300| 220|
|  D-17|    치즈 수플레| 2200| 160|
|  D-18|    호박 타르트| 3400| 230|
|  D-19|      캬라멜 롤| 3700| 230|
+------+---------------+-----+----+
only showing top 20 rows



In [14]:
from pyspark.sql.types import *

In [15]:
orderSchema = StructType([ StructField("orderId", StringType()), \
                           StructField("menuId", StringType()), \
                           StructField("number", IntegerType()) \
    
])

In [17]:
dessertOrderDF = spark.readStream \
.schema(orderSchema) \
.format("csv") \
.load("C:/spark/streamingIn")

In [18]:
dessertOrderDF.printSchema()

root
 |-- orderId: string (nullable = true)
 |-- menuId: string (nullable = true)
 |-- number: integer (nullable = true)



In [21]:
# 주문별 금액

orderTotal = dessertOrderDF.join(dessertMenuDF, dessertOrderDF.menuId == dessertMenuDF.menuId) \
.groupBy(dessertOrderDF.orderId) \
.agg(sum(dessertOrderDF.number * dessertMenuDF.price).alias('orderTotal'))

In [22]:
orderQuery = orderTotal.writeStream \
.format("console") \
.outputMode("complete") \
.start()

In [23]:
type(orderQuery)

pyspark.sql.streaming.StreamingQuery

In [ ]:
orderQuery.awaitTermination()

### Structured Streaming File Source
- 디렉토리에 있는 파일들을 실행
- 새로운 파일이 추가되면 스트리밍 작업을 실행
    - 기존 파일을 포함하여 디렉토리내에 있는 모든 파일을 처리

### Structured Streaming 관련 주제들
- withWatermark("event time","30 minutes")
    - 파일을 읽을 때, 분산환경에서 읽어들이기 때문에 읽은 파일을 제대로 전달받지 못할 가능성이 있음.
        - 얼마나 오래 기다릴 것인가르 지정하는 method임.
        - Structured streaming은 .withWatermark에 지정한 시간이 지날 때까지 기다렸다가, 결과 처리를 종료함.

- dropDuplicates(["User", "event time"])
    - 분산 환경에서는 데이터를 복사하여 다른 경로로 전달할 수 있음
        - 데이터가 중복 전달될 수 있음
    - 인자로 지정한 컬럼들의 값들이 같은 레코드들은 중복된 것으로 간주하여 중복 제거하여 처리

### Windowing: Event-Time
-Processing Time과 Event Time
    - Processing Time: 스트림 데이터 처리 프로그램이 실제로 데이터를 수신한 시각
    - Event Time: 데이터가 발생한 시각
- Dstream에서 스트림 데이터 처리 기준
    - Processing Time
    - 따라서 Window Operation이 가능
- StructuredStreaming에서 스트림 데이터 처리 기준
    - Event Time
    - 각 데이터가 Event time을 나타내는 필드(컬럼)을 가지고 있는 경우 (예:timestamp) 이를 기준으로 groupBy 연산을 통하여 Window Operation을 처리할 수 있음
    
wordCountQuery = wordCount.groupBy(window(wordCount.timestamp, "10 minutes"), wordCount.word).count()

### checkpointLocation 디렉토리의 정보
- 실행된 파일들에 대한 이력정보가 저장되어 있음.
    - 프로그램 개발 중 입력 디렉토리에 있는 파일들을 지우거나 하면, 해당 파일이 없어진 것을 checkpointLocation에 저장된 정보를 사용하여 체크함
    - 따라서, 개발 중에는 사용하지 않는 것이 좋음.

### Structured Streaming 프로그램 개발 시 고려사항
- 입력 파일의 생성 시간이 역순으로 디렉토리에 저장될 경우 ,해당 시간차가 클 경우(즉, 늦게 발생한 파일이 먼저 도착하고, 일찍 발생한 파일이 너무 늦게 도착하면), 늦게 도착한 파일이 처리되지 않음 -> .withWatermark() 참조
- Master 지정
    - loca(*)로 master 지정 -> 속도 개선에 도움이 됨.
        - SparkSession을 생성할 때, master 지정
        - .master("local") -> .master("local[*]")로 지정